In [1]:
import pandas as pd
from gensim import corpora, models
from bs4 import BeautifulSoup

### Parse Contents and extract &lt;Agency> Tags

In [2]:
infile = open("docs/01-3.xml", encoding="utf8")
contents = infile.read()
soup = BeautifulSoup(contents, 'xml')
agencies = soup.find_all('AGENCY')
for agency in agencies:
    print(agency.get_text())

DEPARTMENT OF THE TREASURY 
FEDERAL RESERVE SYSTEM 
FEDERAL DEPOSIT INSURANCE CORPORATION 
DEPARTMENT OF THE TREASURY 


In [12]:
infile = open("docs/01-3.xml", encoding="utf8")
contents = infile.read()
soup = BeautifulSoup(contents, 'xml')
ptags = soup.find_all('P')

lengths = []

for tags in ptags:
    lengths.append(len(tags.get_text()))
    print(tags.get_text())

Office of the Comptroller of the Currency (OCC); Board of Governors of the Federal Reserve System (Board); Federal Deposit Insurance Corporation (FDIC); Office of Thrift Supervision (OTS). 
Joint final rule. 
The OCC, Board, FDIC, and OTS (collectively, the agencies) are publishing final rules to implement the CRA sunshine provisions of section 48 of the Federal Deposit Insurance Act. These provisions require nongovernmental entities or persons (NGEPs), insured depository institutions, and affiliates of insured depository institutions that are parties to certain agreements that are in fulfillment of the Community Reinvestment Act of 1977 to make the agreements available to the public and the appropriate agency and file annual reports concerning the agreements with the appropriate agency. These provisions were contained in section 711 of the Gramm-Leach-Bliley Act. 
The rule identifies the types of written agreements that are covered by section 48 (referred to as covered agreements) and

In [2]:

from gensim import corpora, models

# list_of_list_of_tokens = [["a","b","c"], ["d","e","f"]]
# ["a","b","c"] are the tokens of document 1, ["d","e","f"] are the tokens of document 2...
dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens]

num_topics = 20
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

AttributeError: module 'gensim.corpora' has no attribute 'Dictionary'

#### (3) Find the first &lt;FP> tag with the attribute "SOURCE" having the value "FP-2"

In [3]:
firstTag = soup.find("FP", {"SOURCE" : "FP-2"})
print(firstTag)

<FP SOURCE="FP-2">I. Background </FP>


## Pandas stuff

In [19]:
df = pd.read_csv('financial_services_rules.csv', usecols=
                 ['document_number', 'title', 'type', 'abstract', 'publication_date', 'agencies', 'excerpts'])
df['publication_date']= pd.to_datetime(df['publication_date'])
df = df.sort_values(by='publication_date')
print(df.shape)
df.reset_index(inplace=True, drop=True)
df.head(5)

(3630, 7)


,title,type,abstract,document_number,publication_date,agencies,excerpts
0,Bank Holding Companies and Change in Bank Control,Proposed Rule,The Board of Governors of the Federal Reserve ...,01-43,2001-01-03,"Federal Reserve System, Treasury Department",The Board of Governors of the Federal Reserve ...
1,Regulation Y; Docket Nos. R-1057 and R-1062; B...,Rule,The Board of Governors of the Federal Reserve ...,01-34,2001-01-03,Federal Reserve System,The Board of Governors of the Federal Reserve ...
2,Bank Holding Companies and Change in Bank Control,Rule,The Board of Governors of the Federal Reserve ...,01-42,2001-01-03,"Federal Reserve System, Treasury Department",The Board of Governors of the Federal Reserve ...
3,Activities and Investments of Insured State Banks,Rule,The FDIC is adopting a final rule to implement...,01-175,2001-01-05,Federal Deposit Insurance Corporation,The FDIC is adopting a final rule to implement...
4,Delegation of Authority to Disclose and Reques...,Rule,The Commodity Futures Trading Commission (Comm...,01-595,2001-01-09,Commodity Futures Trading Commission,The Commodity Futures Trading Commission (Comm...


In [20]:
import datetime
df[(df['publication_date']>datetime.datetime(2016,1,1)) & (df['publication_date']<datetime.datetime(2016,3,1))] 

,title,type,abstract,document_number,publication_date,agencies,excerpts
2514,Adoption of Updated EDGAR Filer Manual,Rule,The Securities and Exchange Commission (the Co...,2015-32985,2016-01-04,Securities and Exchange Commission,The Securities and Exchange Commission (the Co...
2515,Margin Requirements for Uncleared Swaps for Sw...,Rule,The Commodity Futures Trading Commission (``Co...,2015-32320,2016-01-06,Commodity Futures Trading Commission,The Commodity Futures Trading Commission (``Co...
2516,Establishing the Form and Manner With Which Se...,Proposed Rule,The Securities and Exchange Commission publish...,2016-00044,2016-01-07,Securities and Exchange Commission,The Securities and Exchange Commission publish...
2517,Alternative to Fingerprinting Requirement for ...,Proposed Rule,The Commodity Futures Trading Commission (``Co...,2016-00045,2016-01-12,Commodity Futures Trading Commission,The Commodity Futures Trading Commission (``Co...
2518,Regulatory Publication and Review Under the Ec...,Proposed Rule,NaN,C1-2015-32312,2016-01-14,"Treasury Department, Comptroller of the Curren...",NaN
2519,Simplification of Disclosure Requirements for ...,Rule,The Securities and Exchange Commission (``Comm...,2016-00872,2016-01-19,Securities and Exchange Commission,The Securities and Exchange Commission (``Comm...
2520,Access to Data Obtained by Security-Based Swap...,Proposed Rule,The Securities and Exchange Commission (``Comm...,2016-01148,2016-01-21,Securities and Exchange Commission,The Securities and Exchange Commission (``Comm...
2521,Extension of Comment Period for Disclosure of ...,Proposed Rule,The Securities and Exchange Commission is exte...,2016-01545,2016-01-27,Securities and Exchange Commission,The Securities and Exchange Commission is exte...
2522,Regulatory Capital Rules: The Federal Reserve ...,Proposed Rule,The Board is inviting public comment on a poli...,2016-01934,2016-02-03,Federal Reserve System,The Board is inviting public comment on a poli...
2523,Assessments,Proposed Rule,"On July 13, 2015, the FDIC published a notice ...",2016-01448,2016-02-04,Federal Deposit Insurance Corporation,"On July 13, 2015, the FDIC published a notice ..."
